In [16]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.prompts.prompt import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
import langchain_core

from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage

from langchain.pydantic_v1 import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser

import os


In [28]:
arquivos_pdf = []
for root, dirs, files in os.walk("./docs"):
    for file in files:
        if file.endswith('.pdf'):
            arquivos_pdf.append(os.path.join(root, file))
print(arquivos_pdf)

['./docs\\Tratamento completo - PDF - final.pdf', './docs\\AnnaPegova\\Descritivo Produtos - Home Care - WhatsApp - Anna Pegova.pdf', './docs\\AnnaPegova\\Manual da Marca Anna Pegova - 2021.pdf', './docs\\Preco\\Linha Profissional Composição - Fev 24.pdf', './docs\\Preco\\Listas de Preço Atualizadas - OUTUBRO 2023.pdf', './docs\\Produtos\\Manual de Produtos 2023.pdf', './docs\\Produtos\\PROTOCOLOS SOS EPIGEN_Est.pdf', './docs\\Produtos\\TOP 20 PRODUTOS - versão pdf.pdf']


In [17]:
class UserData(BaseModel):
    name: str = Field(description="name of the user")
    email: str = Field(description="email of the user")

parser = PydanticOutputParser(pydantic_object=UserData)

In [18]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="Você é um assistente que interage com o usuário de forma amigável e educada e traduz o texto para inglês somente quando o usuário pedir para você fazer isso. Você pode responder perguntas, contar piadas, contar histórias, entre outras coisas. Você é um assistente de linguagem que ajuda o usuário a se comunicar melhor. Sempre que o usuário perguntar explicitamente o nome dele, o telefone dele ou o email dele e você souber a resposta informe da maneira mais simplis possível essas informações sem palavras ou frases adicionais, caso não saiba informe que não possui tais informações e peça para que ele te informe."
        ),  # The persistent system prompt
        MessagesPlaceholder(
            variable_name="chat_history"
        ),  # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template(
            "{input}"
        ),  # Where the human input will injected
    ]
)
prompt.partial_variables = {"format_instructions":parser.get_format_instructions()}

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [19]:
llm = ChatOpenAI()

In [20]:
chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [22]:
output = chat_llm_chain.invoke(input="Olá, meu nome é Cadu e meu email é cccc@hotmail.com, tudo bem com você?")



> Entering new LLMChain chain...
Prompt after formatting:
System: Você é um assistente que interage com o usuário de forma amigável e educada e traduz o texto para inglês somente quando o usuário pedir para você fazer isso. Você pode responder perguntas, contar piadas, contar histórias, entre outras coisas. Você é um assistente de linguagem que ajuda o usuário a se comunicar melhor. Sempre que o usuário perguntar explicitamente o nome dele, o telefone dele ou o email dele e você souber a resposta informe da maneira mais simplis possível essas informações sem palavras ou frases adicionais, caso não saiba informe que não possui tais informações e peça para que ele te informe.
Human: Olá, meu nome é Cadu e meu email é cccc@hotmail.com, tudo bem com você?
AI: Olá Cadu! 😊
Você poderia me informar seu número de telefone, por favor?
Human: Olá, meu nome é Cadu e meu email é cccc@hotmail.com, tudo bem com você?

> Finished chain.


In [26]:
output = chat_llm_chain.invoke(input="Qual meu nome e meu email?")
parser.invoke(output)



> Entering new LLMChain chain...
Prompt after formatting:
System: Você é um assistente que interage com o usuário de forma amigável e educada e traduz o texto para inglês somente quando o usuário pedir para você fazer isso. Você pode responder perguntas, contar piadas, contar histórias, entre outras coisas. Você é um assistente de linguagem que ajuda o usuário a se comunicar melhor. Sempre que o usuário perguntar explicitamente o nome dele, o telefone dele ou o email dele e você souber a resposta informe da maneira mais simplis possível essas informações sem palavras ou frases adicionais, caso não saiba informe que não possui tais informações e peça para que ele te informe.
Human: Olá, meu nome é Cadu e meu email é cccc@hotmail.com, tudo bem com você?
AI: Olá Cadu! 😊
Você poderia me informar seu número de telefone, por favor?
Human: Olá, meu nome é Cadu e meu email é cccc@hotmail.com, tudo bem com você?
AI: Olá Cadu! 😊
Você poderia me informar seu número de telefone, por favor?
Human

ValidationError: 1 validation error for Generation
text
  str type expected (type=type_error.str)

In [6]:
chat_llm_chain.invoke(input="Qual o meu nome?")



> Entering new LLMChain chain...
Prompt after formatting:
System: Você é um assistente que interage com o usuário de forma amigável e educada e traduz o texto para inglês somente quando o usuário pedir para você fazer isso. Você pode responder perguntas, contar piadas, contar histórias, entre outras coisas. Você é um assistente de linguagem que ajuda o usuário a se comunicar melhor. Sempre que o usuário perguntar o nome dele, o telefone dele ou o email dele e você souber a resposta informe da maneira mais simplis possível essas informações sem palavras ou frases adicionais, caso não saiba informe que não possui tais informações e peça para que ele te informe.
Human: Olá, sou o Cadu, tudo bem?
AI: Olá, Cadu! Tudo bem sim, e contigo? Como posso te ajudar hoje?
Human: Qual o meu nome?

> Finished chain.


{'input': 'Qual o meu nome?',
 'chat_history': [HumanMessage(content='Olá, sou o Cadu, tudo bem?'),
  AIMessage(content='Olá, Cadu! Tudo bem sim, e contigo? Como posso te ajudar hoje?'),
  HumanMessage(content='Qual o meu nome?'),
  AIMessage(content='Cadu')],
 'text': 'Cadu'}

In [7]:
chat_llm_chain.invoke(input="Traduza a seguinte frase: Hoje o dia amanheceu nublado e provavelemente choverá.")



> Entering new LLMChain chain...
Prompt after formatting:
System: Você é um assistente que interage com o usuário de forma amigável e educada e traduz o texto para inglês somente quando o usuário pedir para você fazer isso. Você pode responder perguntas, contar piadas, contar histórias, entre outras coisas. Você é um assistente de linguagem que ajuda o usuário a se comunicar melhor.
Human: Olá, sou o Cadu, tudo bem?
AI: Olá Cadu! Estou bem, obrigado por perguntar. Como posso te ajudar hoje?
Human: Qual o meu nome?
AI: Seu nome é Cadu. Posso te ajudar com mais alguma coisa?
Human: Traduza a seguinte frase: Hoje o dia amanheceu nublado e provavelemente choverá.

> Finished chain.


{'input': 'Traduza a seguinte frase: Hoje o dia amanheceu nublado e provavelemente choverá.',
 'chat_history': [HumanMessage(content='Olá, sou o Cadu, tudo bem?'),
  AIMessage(content='Olá Cadu! Estou bem, obrigado por perguntar. Como posso te ajudar hoje?'),
  HumanMessage(content='Qual o meu nome?'),
  AIMessage(content='Seu nome é Cadu. Posso te ajudar com mais alguma coisa?'),
  HumanMessage(content='Traduza a seguinte frase: Hoje o dia amanheceu nublado e provavelemente choverá.'),
  AIMessage(content='Claro! Aqui está a tradução da frase para o inglês: "Today the day dawned cloudy and it will probably rain." \n\nPrecisa de mais alguma tradução ou de outra ajuda?')],
 'text': 'Claro! Aqui está a tradução da frase para o inglês: "Today the day dawned cloudy and it will probably rain." \n\nPrecisa de mais alguma tradução ou de outra ajuda?'}

In [10]:
chat_llm_chain.memory.chat_memory.messages

[HumanMessage(content='Olá, sou o Cadu, tudo bem?'),
 AIMessage(content='Olá Cadu! Estou bem, obrigado por perguntar. Como posso te ajudar hoje?'),
 HumanMessage(content='Qual o meu nome?'),
 AIMessage(content='Seu nome é Cadu. Posso te ajudar com mais alguma coisa?'),
 HumanMessage(content='Traduza a seguinte frase: Hoje o dia amanheceu nublado e provavelemente choverá.'),
 AIMessage(content='Claro! Aqui está a tradução da frase para o inglês: "Today the day dawned cloudy and it will probably rain." \n\nPrecisa de mais alguma tradução ou de outra ajuda?')]